# Neural Network to predict Solar output

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd



In [21]:
# path = '/Users/dltc2020/Documents/Senior-project/Final_Merged_Energy_and_Weather_Data.csv'
path = '/Users/ben/Desktop/Senior-project/Final_Merged_Energy_and_Weather_Data.csv'
data = pd.read_csv(path)

data['solar_output_lag1'] = data['Solar'].shift(1)

data['Previous_Day'] = data['BeginDate'] - pd.Timedelta(days=1)
data = data[['BeginDate', 'Solar']].copy()

def get_previous_day_solar(row, reference_df):
    closest_entry = reference_df.iloc[(reference_df['BeginDate'] - row['Previous_Day']).abs().argsort()[:1]]
    if closest_entry.empty:
        return None
    else:
        return closest_entry['Solar'].values[0]
 

TypeError: unsupported operand type(s) for -: 'str' and 'Timedelta'

In [19]:
data['BeginDate'] = pd.to_datetime(data['BeginDate'])
features = ['cloudcover', 'uvindex', 'visibility', 'sealevelpressure', 'solarradiation']
clean_data = data.dropna(subset=features + ['solarenergy']+['Solar'])

train_data = clean_data[data['BeginDate'].dt.day <= 28].copy()
test_data = clean_data[data['BeginDate'].dt.day >= 29].copy()

train_data['Previous_Day_Solar'] = train_data.apply(get_previous_day_solar, axis=1, reference_df=data)
test_data['Previous_Day_Solar'] = test_data.apply(get_previous_day_solar, axis=1, reference_df=data)

train_data.dropna(subset=['solar_output_lag1', 'Previous_Day_solar'], inplace=True)
test_data.dropna(subset=['solar_output_lag1', 'Previous_Day_solar'], inplace=True)


/var/folders/bl/pt11zfbj2bd16s363ftmwj_h0000gn/T/ipykernel_34274/964468603.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['BeginDate'] = pd.to_datetime(data['BeginDate'])
/var/folders/bl/pt11zfbj2bd16s363ftmwj_h0000gn/T/ipykernel_34274/964468603.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  train_data = clean_data[data['BeginDate'].dt.day <= 28].copy()
/var/folders/bl/pt11zfbj2bd16s363ftmwj_h0000gn/T/ipykernel_34274/964468603.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  test_data = clean_data[data['BeginDate'].dt.day >= 29].copy()


KeyError: 'Previous_Day'

In [10]:
X_train = train_data[features+ ['solarenergy']]
Y_train = train_data['Solar']
X_test = test_data[features+ ['solarenergy']]
Y_test = test_data['Solar']

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(X_train_scaled.shape)
print(Y_train.shape)


(5513, 6)
(5513,)


In [11]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')

history = model.fit(X_train_scaled, Y_train, epochs=20, batch_size=32, validation_split=0.2, verbose=1)
test_loss = model.evaluate(X_test_scaled, Y_test, verbose=1)
print(f"loss: {test_loss}")

predictions = model.predict(X_test_scaled)

# Create a DataFrame with actual and predicted values
results_df = pd.DataFrame({
    'Actual Solar Value': Y_test,
    'Predicted Solar Value': predictions.flatten()
})

# Save the DataFrame to a CSV file
results_df.to_csv('solar_predictions.csv', index=False)

Epoch 1/20
138/138 [==============================] - 1s 3ms/step - loss: 117633.7656 - val_loss: 147308.0469
Epoch 2/20
138/138 [==============================] - 0s 2ms/step - loss: 61443.4766 - val_loss: 67426.0000
Epoch 3/20
138/138 [==============================] - 0s 3ms/step - loss: 27189.4688 - val_loss: 51081.0781
Epoch 4/20
138/138 [==============================] - 0s 3ms/step - loss: 23164.2207 - val_loss: 42614.1797
Epoch 5/20
138/138 [==============================] - 0s 2ms/step - loss: 20702.8945 - val_loss: 38008.2383
Epoch 6/20
138/138 [==============================] - 1s 4ms/step - loss: 19005.5586 - val_loss: 34976.0938
Epoch 7/20
138/138 [==============================] - 0s 3ms/step - loss: 17481.0117 - val_loss: 32628.4746
Epoch 8/20
138/138 [==============================] - 0s 3ms/step - loss: 16067.5850 - val_loss: 30324.7891
Epoch 9/20
138/138 [==============================] - 0s 2ms/step - loss: 14764.6113 - val_loss: 28291.2363
Epoch 10/20
138/138 [=====

In [1]:
f = open('/Users/ben/Desktop/Senior-project/TheMotherLoad.csv', 'r')
data = f.readlines()

data[:10][:]


['BeginDate,Coal,Hydro,Natural Gas,Nuclear,Oil,Other,Landfill Gas,Refuse,Solar,Wind,Wood,datetime,temp,dew,humidity,precip,preciptype,snow,snowdepth,windgust,windspeed,winddir,sealevelpressure,cloudcover,solarradiation,solarenergy,uvindex,severerisk\n',
 '2023-10-01 04:01:10+00:00,0.0,581.0,5544.0,3035.0,0.0,10.0,6.0,322.0,1.0,118.0,123.0,2023-10-01 04:00:00+00:00,54.9,48.8,79.8,0.0,,0.0,0,6.9,7.8,331,1020.9,100.0,0,0.0,0,10\n',
 '2023-10-01 04:11:17+00:00,0.0,579.0,5334.0,3031.0,0.0,17.0,6.0,313.0,1.0,115.0,126.0,2023-10-01 04:00:00+00:00,54.9,48.8,79.8,0.0,,0.0,0,6.9,7.8,331,1020.9,100.0,0,0.0,0,10\n',
 '2023-10-01 04:16:03+00:00,0.0,582.0,5297.0,3031.0,0.0,27.0,6.0,315.0,1.0,116.0,128.0,2023-10-01 04:00:00+00:00,54.9,48.8,79.8,0.0,,0.0,0,6.9,7.8,331,1020.9,100.0,0,0.0,0,10\n',
 '2023-10-01 04:16:15+00:00,0.0,582.0,5297.0,3031.0,0.0,27.0,6.0,315.0,1.0,116.0,128.0,2023-10-01 04:00:00+00:00,54.9,48.8,79.8,0.0,,0.0,0,6.9,7.8,331,1020.9,100.0,0,0.0,0,10\n',
 '2023-10-01 04:19:58+00:00,0.